<a href="https://colab.research.google.com/github/kuostar0620-jpg/114-1KUO-REPO-/blob/main/41371124hweek5%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================
# PTT -> Google Sheet -> TF-IDF -> Gemini -> Gradio
# 完整自動化範例 (Google Colab)
# ======================

# 安裝套件（Colab 執行）
# 請確認這些套件已安裝，Colab 環境可能需要額外安裝 google-auth
# !pip install -q requests beautifulsoup4 pandas jieba wordcloud tqdm sklearn gspread oauth2client gradio google-auth requests_oauthlib

# ----------------------
# 匯入
# ----------------------
import os
import time
import json
import requests
import pandas as pd
import jieba
from bs4 import BeautifulSoup
from collections import Counter
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import io # 用於處理 matplotlib 圖片

# Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Gradio
import gradio as gr

# ----------------------
# 使用者需先準備：service_account.json 並上傳到 Colab
# 並準備好 Google Sheet 授權讓該服務帳號有編輯權
# 將下面的 SHEET_ID/API_KEY 改成你的設定
# ----------------------
SERVICE_ACCOUNT_FILE = "service_account.json" # 服務帳號 JSON 檔案名稱
SHEET_ID = "1eTIiO-nrJWfa9Oi61BqryQRSKQAOg73MHKv2dK5_yjw" # <--- 已更新為您的 Google Sheet ID
GEMINI_MODEL_NAME = "gemini-2.5-flash-preview-09-2025"
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY" # <--- 改成你的 Gemini API Key

# ----------------------
# 設定 PTT 爬蟲參數
# ----------------------
BASE_URL = "https://www.ptt.cc"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
# 設置 over18 cookie 以繞過年齡限制
cookies = {"over18": "1"}
# 預設爬取看板與頁數
DEFAULT_BOARD = "Stock"
DEFAULT_PAGES = 3

# ----------------------
# 輔助函數：Google Sheets 連線
# ----------------------
def init_gsheets(service_account_file=SERVICE_ACCOUNT_FILE):
    """初始化 Google Sheets 連線與授權"""
    try:
        scope = [
            "https://spreadsheets.google.com/feeds",
            "https://www.googleapis.com/auth/drive",
        ]
        # 建立憑證
        creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_file, scope)
        client = gspread.authorize(creds)
        sheet = client.open_by_key(SHEET_ID)
        return sheet
    except Exception as e:
        print(f"Google Sheets 初始化失敗，請檢查 service_account.json 和 SHEET_ID: {e}")
        raise

# ----------------------
# 輔助函數：PTT 爬蟲 - 取得文章清單
# ----------------------
def get_articles_from_board(board_name, num_pages=1):
    """從指定看板爬取指定頁數的文章清單和連結"""
    articles_data = []
    current_url = f"{BASE_URL}/bbs/{board_name}/index.html"

    # 查找前 N 頁的 index
    page_urls = []
    for _ in tqdm(range(num_pages), desc="尋找頁面"):
        res = requests.get(current_url, headers=headers, cookies=cookies, timeout=10)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")

        # 找到「上一頁」連結，以獲得前一頁的 URL
        prev_url_tag = soup.select('div.btn-group-paging a')[1]
        prev_url = BASE_URL + prev_url_tag['href']
        page_urls.append(current_url)
        current_url = prev_url

    # 從頁面清單中抓取文章
    for page_url in tqdm(page_urls, desc="爬取文章清單"):
        res = requests.get(page_url, headers=headers, cookies=cookies, timeout=10)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")

        for item in soup.select(".r-ent"):
            try:
                title_tag = item.select_one(".title a")
                meta_tag = item.select_one(".meta")

                if title_tag and meta_tag:
                    title = title_tag.text.strip()
                    link = BASE_URL + title_tag["href"]
                    date = meta_tag.select_one(".date").text.strip()
                    author = meta_tag.select_one(".author").text.strip()

                    # 過濾掉被刪除的文章
                    if "已被刪除" not in title:
                        articles_data.append({
                            "看板": board_name,
                            "標題": title,
                            "連結": link,
                            "日期": date,
                            "作者": author,
                            "內文": "" # 預留欄位
                        })
            except Exception as e:
                # 忽略異常的文章項目
                # print(f"處理文章清單時發生錯誤: {e}")
                continue

        time.sleep(0.5) # 避免過快被擋

    return articles_data

# ----------------------
# 輔助函數：PTT 爬蟲 - 取得單篇文章內文
# ----------------------
def get_article_content(link):
    """取得單篇文章的純文字內文"""
    try:
        res = requests.get(link, headers=headers, cookies=cookies, timeout=10)
        res.encoding = "utf-8"
        soup = BeautifulSoup(res.text, "html.parser")

        main_content = soup.find(id="main-content")

        if not main_content:
            return ""

        # 移除簽名檔、推文等雜訊
        for tag in main_content.find_all(class_=["f2", "f4", "push"]):
            tag.decompose()

        # 移除 meta 資訊 (作者, 看板, 標題, 時間)
        for meta in main_content.find_all(class_="article-meta-value"):
            meta.decompose()

        # 移除所有空白行和多餘的換行
        content = main_content.text.strip()

        return content
    except Exception:
        return ""

# ----------------------
# 核心函數：TF-IDF 關鍵字分析
# ----------------------
# 簡單的中文停用詞列表
STOPWORDS = [
    '的', '是', '在', '我', '你', '他', '她', '它', '們', '這', '那', '個',
    '了', '和', '有', '都', '會', '要', '也', '而', '之', '所', '來', '去',
    '嗎', '啊', '喔', '咧', '啦', '吧', '對', '跟', '就', '很', '可以',
    '一個', '一個', '可以', '說', '想', '覺得', '還有', '什麼', '這樣', '因為', '所以'
]

def analyze_tfidf(df_articles, top_n=30):
    """
    執行中文分詞和 TF-IDF 關鍵字提取
    :param df_articles: 包含 '內文' 欄位的 DataFrame
    :param top_n: 提取前 N 個關鍵字
    :return: 包含關鍵字和 TF-IDF 分數的 DataFrame
    """
    # 進行分詞
    df_articles['分詞內文'] = df_articles['內文'].apply(
        lambda x: " ".join([word for word in jieba.cut(x) if word.strip() and word not in STOPWORDS and len(word) > 1])
    )

    documents = df_articles['分詞內文'].tolist()

    if not documents:
        return pd.DataFrame({'關鍵字': [], 'TF-IDF 分數': []})

    # TF-IDF 計算
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform(documents)
    except ValueError:
        return pd.DataFrame({'關鍵字': [], 'TF-IDF 分數': []}) # 資料不足時

    feature_names = vectorizer.get_feature_names_out()

    # 取得每個文件的所有 TF-IDF 值
    all_scores = tfidf_matrix.toarray().sum(axis=0)

    # 整理結果
    results = pd.DataFrame({'關鍵字': feature_names, '總TF-IDF 分數': all_scores})
    results = results.sort_values(by='總TF-IDF 分數', ascending=False).head(top_n)

    return results

# ----------------------
# 核心函數：Gemini API 報告生成
# ----------------------
def generate_insight_report(text_summary, api_key):
    """呼叫 Gemini API 生成洞察摘要與結論"""
    if not api_key or api_key == "YOUR_GEMINI_API_KEY":
        return "Gemini API Key 尚未設定，無法生成報告。"

    if not text_summary:
        return "無文本可供分析。"

    # 設定 System Instruction 和 User Prompt
    system_prompt = (
        "你是一位專業的市場分析師。請根據提供的文本內容，生成一份簡潔的洞察報告。你的輸出必須包含以下兩個部分，並使用繁體中文：\n"
        "1. 五點關鍵洞察摘要（每點一句話，使用編號清單）。\n"
        "2. 一段約120字的綜合結論（總結文本核心主旨和趨勢）。請確保結論的字數在100到130個繁體中文字之間。"
    )

    # 為了節省 token，只傳遞 TF-IDF 提取的關鍵內容
    user_query = f"這是文本分析的關鍵內容：\n\n{text_summary}\n\n請依據要求生成洞察報告。"

    # API 請求設定
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_MODEL_NAME}:generateContent?key={api_key}"

    payload = {
        "contents": [{"parts": [{"text": user_query}]}],
        "systemInstruction": {"parts": [{"text": system_prompt}]},
        "config": {
            # 調整溫度，讓報告更具分析性
            "temperature": 0.6,
        }
    }

    try:
        # 實作指數退避 (Exponential Backoff) 處理
        max_retries = 3
        for attempt in range(max_retries):
            response = requests.post(
                url,
                headers={'Content-Type': 'application/json'},
                data=json.dumps(payload),
                timeout=30
            )

            if response.status_code == 200:
                break

            # 對 429 (Too Many Requests) 或 5xx 錯誤進行重試
            if response.status_code in [429, 500, 503] and attempt < max_retries - 1:
                wait_time = 2 ** attempt # 1s, 2s, 4s
                time.sleep(wait_time)
                continue

            response.raise_for_status() # 檢查 HTTP 錯誤

        result = response.json()

        # 解析結果
        text = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'API 返回內容解析失敗。')
        return text

    except requests.exceptions.RequestException as e:
        return f"Gemini API 呼叫失敗: {e}"
    except Exception as e:
        return f"發生未知錯誤: {e}"

# ----------------------
# 整合函數：批次執行所有流程 (寫入 Google Sheet)
# ----------------------
def run_all_analysis(board_name, num_pages, gemini_api_key):
    """
    執行批次爬蟲、內文抓取、寫入 Sheet、TF-IDF 分析、回寫 Sheet、Gemini 報告生成
    並返回 Gradio 所需的輸出
    """

    # 檢查必要設定
    if SHEET_ID == "YOUR_GOOGLE_SHEET_ID_HERE":
        return "錯誤：請將 SHEET_ID 替換為您的 Google Sheet ID。", pd.DataFrame(), pd.DataFrame(), None, "分析失敗。"

    # 1. 初始化 Google Sheets
    try:
        sheet = init_gsheets()
        worksheet_articles = sheet.worksheet("PTT 文章清單")
        worksheet_keywords = sheet.worksheet("關鍵字統計")

        # 清空工作表 (避免資料重複)
        worksheet_articles.clear()
        worksheet_keywords.clear()

    except Exception as e:
        return f"Google Sheets 連線或工作表操作失敗: {e}", pd.DataFrame(), pd.DataFrame(), None, "分析失敗。"

    # 2. PTT 爬蟲：抓取文章清單
    gr.Info(f"正在爬取 PTT 看板: {board_name}，共 {num_pages} 頁...")
    articles_data = get_articles_from_board(board_name, num_pages)

    if not articles_data:
        return f"看板 {board_name} 未找到任何文章。", pd.DataFrame(), pd.DataFrame(), None, "分析失敗。"

    df_articles = pd.DataFrame(articles_data)

    # 3. PTT 爬蟲：抓取內文
    for i, row in tqdm(df_articles.iterrows(), total=len(df_articles), desc="抓取文章內文"):
        df_articles.loc[i, '內文'] = get_article_content(row['連結'])
        time.sleep(0.1) # 溫和爬蟲

    # 篩選掉內文為空的文章
    df_articles = df_articles[df_articles['內文'].str.strip() != ''].reset_index(drop=True)

    # 4. 寫入 Google Sheet (文章清單)
    gr.Info("將文章清單寫入 Google Sheets...")
    # 僅寫入需要的欄位
    display_df = df_articles[["看板", "標題", "連結", "日期", "作者", "內文"]]
    worksheet_articles.update([display_df.columns.tolist()] + display_df.values.tolist())

    # 5. TF-IDF 分析
    gr.Info("執行 TF-IDF 關鍵字分析...")
    jieba.set_dictionary('dict.txt.big') # 假設使用內建大詞典
    results_tfidf = analyze_tfidf(df_articles, top_n=50) # 取前 50

    # 6. 寫入 Google Sheet (關鍵字統計)
    gr.Info("將 TF-IDF 結果回寫到 Google Sheets...")
    worksheet_keywords.update([results_tfidf.columns.tolist()] + results_tfidf.values.tolist())

    # 7. 準備 Gemini 輸入文本：使用前 20 個關鍵字作為上下文
    top_keywords_str = results_tfidf.head(20).to_string(header=False, index=False)
    summary_text = f"以下是看板 [{board_name}] 的熱門關鍵字和它們的總 TF-IDF 分數：\n{top_keywords_str}"

    # 8. 呼叫 Gemini API
    gr.Info("呼叫 Gemini API 生成洞察報告...")
    gemini_report = generate_insight_report(summary_text, gemini_api_key)

    # 9. 準備圖表輸出
    chart_output = None
    if not results_tfidf.empty:
        # 繪製詞頻圖
        plt.figure(figsize=(10, 8))
        plt.barh(results_tfidf['關鍵字'].head(20), results_tfidf['總TF-IDF 分數'].head(20), color='skyblue')
        plt.xlabel('總 TF-IDF 分數 (Total TF-IDF Score)')
        plt.title(f'{board_name} 看板熱門關鍵字 (前 20)')
        plt.gca().invert_yaxis()

        # 儲存圖片到緩衝區
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)

        # 關閉圖表以釋放記憶體
        plt.close()

        chart_output = buf.getvalue()

    gr.Info("批次分析流程執行完畢！")

    # 返回 Gradio 元件所需的輸出
    return display_df, results_tfidf, chart_output, gemini_report, "分析成功！請查看 Google Sheet 中的 [PTT 文章清單] 與 [關鍵字統計]。"


# ----------------------
# 整合函數：單篇文章分析 (寫入 Google Sheet)
# ----------------------
def run_single_article_analysis(article_url, gemini_api_key):
    """
    執行單篇文章爬蟲、TF-IDF 分析和 Gemini 報告生成 (並寫入 Google Sheet)
    """
    if SHEET_ID == "YOUR_GOOGLE_SHEET_ID_HERE":
        return pd.DataFrame(), None, "錯誤：請將 SHEET_ID 替換為您的 Google Sheet ID。", "分析失敗。"

    gr.Info(f"正在爬取單篇文章: {article_url}...")

    # 1. 爬取內文
    content = get_article_content(article_url)

    if not content:
        return pd.DataFrame(), None, "無法取得文章內容，請檢查 URL 是否正確或文章是否已被刪除。", "分析失敗。"

    # 2. 準備 DataFrame 進行 TF-IDF
    df_single = pd.DataFrame([{"連結": article_url, "內文": content}])

    # 3. 初始化 Google Sheets (使用新的工作表名稱)
    try:
        sheet = init_gsheets()
        # 嘗試取得工作表，如果不存在則創建
        try:
            worksheet_content = sheet.worksheet("單篇文章內容")
        except gspread.WorksheetNotFound:
            worksheet_content = sheet.add_worksheet(title="單篇文章內容", rows=100, cols=20)

        try:
            worksheet_keywords = sheet.worksheet("單篇文章關鍵字")
        except gspread.WorksheetNotFound:
            worksheet_keywords = sheet.add_worksheet(title="單篇文章關鍵字", rows=100, cols=20)

        # 清空工作表
        worksheet_content.clear()
        worksheet_keywords.clear()

    except Exception as e:
        return pd.DataFrame(), None, f"Google Sheets 連線或工作表操作失敗: {e}", "分析失敗。"

    # 4. 寫入 Google Sheet (單篇文章內容)
    gr.Info("將單篇文章內容寫入 Google Sheets...")
    worksheet_content.update([df_single.columns.tolist()] + df_single.values.tolist())

    # 5. TF-IDF 分析
    gr.Info("執行 TF-IDF 關鍵字分析...")
    jieba.set_dictionary('dict.txt.big')
    # 只需要針對單篇文章進行 TF-IDF，top_n 取 30
    results_tfidf = analyze_tfidf(df_single, top_n=30)

    # 6. 寫入 Google Sheet (關鍵字統計)
    gr.Info("將 TF-IDF 結果回寫到 Google Sheets...")
    worksheet_keywords.update([results_tfidf.columns.tolist()] + results_tfidf.values.tolist())

    # 7. 準備 Gemini 輸入文本
    top_keywords_str = results_tfidf.to_string(header=False, index=False)
    summary_text = f"以下是單篇文章 ({article_url}) 的熱門關鍵字和它們的總 TF-IDF 分數：\n{top_keywords_str}"

    # 8. 呼叫 Gemini API
    gr.Info("呼叫 Gemini API 生成洞察報告...")
    gemini_report = generate_insight_report(summary_text, gemini_api_key)

    # 9. 準備圖表輸出
    chart_output = None
    if not results_tfidf.empty:
        # 繪製詞頻圖
        plt.figure(figsize=(10, 8))
        plt.barh(results_tfidf['關鍵字'].head(20), results_tfidf['總TF-IDF 分數'].head(20), color='lightcoral') # 換個顏色區分
        plt.xlabel('總 TF-IDF 分數 (Total TF-IDF Score)')
        plt.title('單篇文章關鍵字 (前 20)')
        plt.gca().invert_yaxis()

        # 儲存圖片到緩衝區
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close()

        chart_output = buf.getvalue()

    gr.Info("單篇文章分析完畢！")

    # 返回 Gradio 元件所需的輸出: 關鍵字表格, 圖表, Gemini 報告, 狀態訊息
    return results_tfidf, chart_output, gemini_report, "分析成功！單篇文章內容與關鍵字已寫入 Google Sheet 中的 [單篇文章內容] 和 [單篇文章關鍵字]。"


# ----------------------
# Gradio 介面設定
# ----------------------

with gr.Blocks(title="PTT 自動化內容分析器") as demo:
    gr.Markdown(
        """
        # PTT 熱門話題 自動化分析與洞察報告 (Gemini)
        這個工具提供兩種模式：批次看板分析（多篇文章）和單篇文章分析。
        所有結果都會寫入您的 Google Sheet 中。
        """
    )

    with gr.Tabs():
        # --- 批次看板分析 Tab ---
        with gr.TabItem("批次看板分析 (多頁多文)"):
            with gr.Row():
                with gr.Column(scale=1):
                    # 輸入欄位
                    input_board = gr.Textbox(label="PTT 看板名稱", value=DEFAULT_BOARD, info="例如：Stock, Gossiping, Movie")
                    input_pages = gr.Slider(label="爬取頁數", minimum=1, maximum=20, value=DEFAULT_PAGES, step=1, info="建議不要超過 5 頁以避免超時")
                    input_api_key_batch = gr.Textbox(label="Gemini API Key", value=GEMINI_API_KEY, type="password", info="請確保您的金鑰已填寫")

                    # 執行按鈕
                    run_button_batch = gr.Button("🚀 一鍵執行批次分析與報告生成", variant="primary")

                    # 狀態回饋
                    output_status_batch = gr.Textbox(label="狀態/訊息", value="請設定參數後點擊執行。", interactive=False)

                with gr.Column(scale=2):
                    # 輸出區域
                    output_report_batch = gr.Markdown("## Gemini 洞察報告\n\n報告生成後將顯示於此...", label="Gemini 洞察報告")

            gr.Markdown("---")

            with gr.Tabs():
                with gr.TabItem("文章清單與內文"):
                    output_articles_df = gr.Dataframe(
                        label="爬取到的文章清單 (已寫入 Google Sheet [PTT 文章清單])",
                        headers=["看板", "標題", "連結", "日期", "作者", "內文"],
                        datatype=["str", "str", "str", "str", "str", "str"],
                        wrap=True
                        # 已移除 max_rows 參數，以避免 TypeError
                    )

                with gr.TabItem("關鍵字分析結果"):
                    output_keywords_df = gr.Dataframe(
                        label="TF-IDF 關鍵字統計 (已寫入 Google Sheet [關鍵字統計])",
                        headers=["關鍵字", "總TF-IDF 分數"],
                        datatype=["str", "number"],
                        wrap=True
                    )
                    output_chart_batch = gr.Plot(label="熱門關鍵字分佈圖")

            # 綁定按鈕與函數
            run_button_batch.click(
                fn=run_all_analysis,
                inputs=[input_board, input_pages, input_api_key_batch],
                outputs=[output_articles_df, output_keywords_df, output_chart_batch, output_report_batch, output_status_batch]
            )

        # --- 單篇文章分析 Tab ---
        with gr.TabItem("單篇文章分析 (寫入 Sheet)"):
            with gr.Row():
                with gr.Column(scale=1):
                    # 輸入欄位
                    # 預設值使用您提供的網址
                    input_url = gr.Textbox(label="PTT 文章完整 URL", value="https://www.ptt.cc/bbs/movie/M.1761461472.A.C84.html", info="請輸入完整的 PTT 網址，如：https://www.ptt.cc/bbs/movie/M.xxx.html")
                    input_api_key_single = gr.Textbox(label="Gemini API Key", value=GEMINI_API_KEY, type="password", info="請確保您的金鑰已填寫")

                    # 執行按鈕
                    run_button_single = gr.Button("🔍 分析單篇文章並寫入 Sheet", variant="stop")

                    # 狀態回饋
                    output_status_single = gr.Textbox(label="狀態/訊息", value="請輸入 URL 後點擊執行。結果將寫入 [單篇文章內容] 和 [單篇文章關鍵字]。", interactive=False)

                with gr.Column(scale=2):
                    # 輸出區域
                    output_report_single = gr.Markdown("## Gemini 洞察報告\n\n報告生成後將顯示於此...", label="Gemini 洞察報告")

            gr.Markdown("---")

            output_keywords_df_single = gr.Dataframe(
                label="TF-IDF 關鍵字統計 (已寫入 Google Sheet [單篇文章關鍵字])",
                headers=["關鍵字", "總TF-IDF 分數"],
                datatype=["str", "number"],
                wrap=True
            )
            output_chart_single = gr.Plot(label="單篇文章關鍵字分佈圖")

            # 綁定按鈕與函數
            run_button_single.click(
                fn=run_single_article_analysis,
                inputs=[input_url, input_api_key_single],
                outputs=[output_keywords_df_single, output_chart_single, output_report_single, output_status_single]
            )

# 啟動 Gradio 介面
if __name__ == "__main__":
    # 假設在本地運行，如果在 Colab 中，可以設定 share=True
    # demo.launch(share=True)
    demo.launch()

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eea9cc587716f24319.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
